# Graph runbook 1

In [ ]:
%%bash
docker run --name age -d -e POSTGRES_HOST_AUTH_METHOD=trust -p 5432:5432 postgres:16.2
sleep 2

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE TABLE node (
    id BIGSERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    species TEXT NOT NULL
);

CREATE TABLE edge (
    id BIGSERIAL PRIMARY KEY,
    source BIGINT REFERENCES node(id),
    target BIGINT REFERENCES node(id),
    property TEXT NOT NULL
);
"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

INSERT INTO node (name, species)
    VALUES  ('Vemund Santi', 'Human'),
            ('Kristian Vikenes', 'Human'),
            ('Trine Hansen', 'Human'),
            ('Barten', 'Cat'),
            ('Yava', 'Dog');

INSERT INTO edge (source, target, property)
    VALUES  (1, 2, 'friend_with'),
            (1, 3, 'friend_with'),
            (3, 4, 'owner_of'),
            (2, 5, 'owner_of');
"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

-- Find all friends of Vemund Santi

SELECT friend.name
FROM node
    JOIN edge ON node.id = edge.source AND edge.property = 'friend_with'
    JOIN node AS friend ON friend.id = edge.target
WHERE node.name = 'Vemund Santi';
"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

-- Find all friends of Vemund Santi that owns a cat

SELECT friend.name
FROM node
    JOIN edge ON node.id = edge.source AND edge.property = 'friend_with'
    JOIN node AS friend ON friend.id = edge.target
    JOIN edge AS owner ON owner.source = friend.id AND owner.property = 'owner_of'
    JOIN node AS pet ON pet.id = owner.target
WHERE node.name = 'Vemund Santi' AND pet.species = 'Cat';

"

In [ ]:
%%bash
docker stop age
docker rm age